## Learning Realistic Shapelets

Shapelet-based techniques can roughly be divided up into the following categories. Shapelet Discovery approaches [1, 2, 3, 4, 5, 6], Learning Shapelets approaches [7, 8], and hybrid approaches [9, 10] whereas hybrid approaches combine ideas from the other two categories. The learning shapelets approach shows a good trade-off between speed and accuracy. One reason for that might be its capability to learn shapelets, which must not be directly part of the time-series dataset. Since shapelet discovery approaches use subsequences from the dataset, they do not offer such a capability. Unfortunately, this extra capability of the learning shapelets approaches can lead to learning of shapelets, which poorly approximate patterns found in the dataset. Shapelets which poorly approximate patterns of the dataset loose their interpretability. On the contrary, we refer to shapelets as 'realistic' if they closely approximate actual patterns in the dataset. Shapelet discovery approaches by definition fullfil the property to be realistic because the shapelets are directly sampled from the dataset. Figure 1 shows a sample of learned shapelets with *Learning Shapelets* [7] on the left, which illustrates the difficulty of the appoach to come up with realistic shapelets. On the right, an alternative loss function $\mathcal{L_R}$ (which we will introduce below) forcing the shapelets to better approximate patterns of the dataset. 

![shapelet_improvements](images/stats/learning_shapelets_vs_adapted_loss.png)

**Figure 1**: Some of the learned shapelets (orange) overlaid on its best matching time series (blue).  
*Left*: Best run of *Learning Shapelets* [7] with 91.6% accuracy.  
*Right*: Best run with the loss $\mathcal{L_R}$ ($\lambda_1=0.1$, $\lambda_2=0.01$, $k=0.1\cdot batch\_size$) with
93.4% accuracy.  
The patterns learned with $\mathcal{L_R}$ resemble
much more closely actual patterns of the data set.  
Dataset: UWaveGestureLibraryAll

To alleviate the difficulty of the *Learning Shapelets* approach to produce realistic shapelets, Wang et al. [8] propose "Learning Interpretable Shapelets through  Adversarial Regularization". The approach uses an additional network with a similar structure than the *Learning Shapelets* model, which shall serve as a judge to evaluate if the learned shapelets resemble patterns from the data. The two networks share the parameters for the shapelets and are trained alternatingly. We found a simpler way to force *Learning Shapelets* to learn more realistic patterns via simpler regularization terms. A very similar regularization was found by Kidger et al. in [11]. The loss $\mathcal{L_R}$ is introduced hereafter.

## Adapted Loss Function

The following shows the adapted loss function leading to more realistic shapelets.

\begin{equation}
    \mathcal{L_R} = \mathcal{L}_{ce} + \lambda_1 \cdot \mathcal{L}_{d_k} + \lambda_2 \cdot \mathcal{L}_{ss}
\end{equation}

where 

1) $\mathcal{L}_{ce}$ is the cross-entropy loss

\begin{equation}
    \mathcal{L}_{ce}(\text{TS}) = - \sum\limits_{\text{ts} \in \text{TS}} \sum\limits_{\text{c} \in \text{C}} I_\text{C}(\text{c}) \cdot \log(p(\text{ts}))
\end{equation} 
with time series $\text{TS}$ of a batch, $dist(x_1, x_2)$ the distance (in our experiments the euclidean distance) between a shapelet $s$ and a time series $\text{ts}$ (captured by the shapelet transform),  $I_\text{C}$ being the indicator function returning 1 if $\text{c}$ matches the class of $\text{ts}$ and $p(\text{ts})$ the output of the softmax layer after the logistic regression layer.

2) $\mathcal{L}_{d_k}$ is a regularization term that tries to minimize the distance of a shapelet to the top k closest time series of the current batch:

\begin{equation}
    \mathcal{L}_{d_k}(\text{S}, \text{TS}_k) = \sum\limits_{s \in S} \sum\limits_{\text{ts} \in \text{TS}_k} dist(s, \text{ts})
\end{equation}

with $\text{TS}_k$ being the top k closest time series of a batch of samples and $\text{s}$ being the shapelets of a block with the same shapelet length.

3) $\mathcal{L}_{ss}$ is another regularization term that tries to minimize the similarity of the shapelets of a class to counter the effects of $\mathcal{L}_{d_k}$ which might result in multiple shapelets looking similar:

\begin{equation}
    \mathcal{L}_{ss} = \sum\limits_{k \in K} \sum\limits_{s_1, s_2\in B_k} \frac{s_1 \cdot s_2}{||s_1|| \text{ } ||s_2||}
\end{equation}

with $\text{K}$ blocks of shapelets.

The two new regularization terms $\mathcal{L}_{d_k}$ and $\mathcal{L}_{ss}$ shall serve the following purposes:
 * $\mathcal{L}_{d_k}$: Force the shapelets to become close to a certain pattern of the dataset. The parameter *k* shall give control to how frequent the pattern shall be.
 * $\mathcal{L}_{ss}$: To counter the effect of $\mathcal{L}_{d_k}$ and by that to avoid that the learned shapelets will become too similar, $\mathcal{L}_{ss}$ shall force the shapelets to become more distinct between each other. 

## Experiments

To evaluate the effects of $\mathcal{L_R}$, we performed experiments on 85 of the UCR datasets [12]. We were running own experiments on our implementation of *Learning Shapelets* (the one published in this repo) and we ran multiple experiments with different regularization strengths for $\lambda_1 = \{0.1, 0.2, 0.4, 0.8\}$. For each loss function, i.e. cross-entropy (original *Learning Shapelets*) and each of the runs of $\mathcal{L_R}$ with different regularization strength, we performed a hyperparameter search over 18 different shapelet configurations. For each of the 18 shapelet configurations, we trained three models.  The 18 shapelet configurations have been derived via

* Number of shapelets $k\in\{4, 8, 16\}$
* Minimal length of the shapelets $l_{min} \in \{0.03\%, 0.2\%\}$
* Scaling of the minimal length $r\in \{1, 2, 3\}$

For example, considering a dataset of time series with 500 time steps, the configuration 

\{$k$: 4, $l_{min}$: 0.2, r: 2\}

amounts to learning 4 shapelets of length 100 and 4 shapelets of length 200. We, so far, did not have the resources to evaluate $\lambda_2$ and set a value of $\lambda_2=0.01$.

To summarize, for each of the 5 different losses

* Learning Shapelets (cross-entropy)
* $\mathcal{L_R}$ with $\lambda_1=0.1$, $\lambda_2=0.01$, and $k=0.1 \cdot \textit{batch\_size}$
* $\mathcal{L_R}$ with $\lambda_1=0.2$, $\lambda_2=0.01$, and $k=0.1 \cdot \textit{batch\_size}$
* $\mathcal{L_R}$ with $\lambda_1=0.4$, $\lambda_2=0.01$, and $k=0.1 \cdot \textit{batch\_size}$
* $\mathcal{L_R}$ with $\lambda_1=0.8$, $\lambda_2=0.01$, and $k=0.1 \cdot \textit{batch\_size}$

we trained $18 \cdot 3 = 54$ models over $85$ datasets amounting to $5 \cdot 54 \cdot 85 = 22950$ trained models in total. Also note, that at maximum 48 shapelets are learned for a single dataset.

Further, we trained without a validation data set and for at most 8000 epochs.
If the training accuracy did not improve more than by at least 0.1% over 200 consecutive epochs, we early stopped the training. For training we used the Adam optimizer [13] with a learning rate of 0.04, betas=(0.9, 0.999), eps=1e-08, and weight_decay=0.0001 (Learning Shapelets)/weight_decay=0 ($\mathcal{L_R}$). All of the shapelets have been initialized via k-Means as was done in the 2014 paper of *Learning Shapelets* [7] and for each shapelet configuration all models have been initialized with the same shapelets. Training was performed on a single NVIDIA Tesla P100.

**NOTE**: There is one exceptional dataset, *ItalyPowerDemand*, for which we conducted experiments manually, since most of the configurations defined by the hyper parameters above did not lead to reasonable results (probably because of the short length of the time series, the optimization process always ended up with exploding gradients). With a little bit of playing around, we found the following configuration for the dataset, which produced good results (the other parameters are as described above):
* *ItalyPowerDemand*:
    * Shapelets: {16: 16}
    * lr: 2e-5

## Results

First, we evaluate the accuracy of $\mathcal{L_R}$ compared to *Learning Shapelets* [7] and the approach of Wang et al. [8]. Both are also learning-based approaches.   

Second, we briefly evaluate the runtime of $\mathcal{L_R}$ against *Learning Shapelets*.

Third, we measure the quality of the learned shapelets by their distance to the top 10 best matching time series of the respective dataset. We can only compare the distances of $\mathcal{L_R}$ to *Learning Shapelets* for which we conducted our own experiments. We did not conduct experiments for the approach of Wang et al. [8] and therefore we do not have shapelet data for their approach to compare the distances to.

### Accuracy

Here, compare the accuracies of $\mathcal{L_R}$ to *Learning Shapelets* [7] and to the approach *Learning Interpretable Shapelets through Adversarial Regularization* of Wang et al. [8].

See Tabel 1 below for a table that compares the accuracies to related work. *Learning Shapelets* [7], *Learning Shapelets with $\mathcal{L_R}$ (l1=0.1, l2=0.01)*, and *Ultra-Fast Shapelets for Time Series Classification* have similar and superior accuracy compared to the rest of the approaches, but *Ultra-Fast Shapelets for Time Series Classification* seems to need many more shapelets.

#### Learning Shapelets vs Adapted Learning Shapelets

![shapelet_acc_ls_dist](images/stats/accuracies_learning_shapelets_vs_dist_loss.png)

**Figure 2**: Each subplot shows a comparison of the best run between the accuracy of
*Learning Shapelets* and learning with $\mathcal{L_R}$ with varying
values for $\lambda_1$. The values on the y-axis correspond to the accuracy of *Learning
Shapelets* and the values on the x-axis to the accuracy of learning with $\mathcal{L_R}$.
The subplots marked with a red cross show a significant decrease in accuracy
compared to *Learning Shapelets*. The arrow indicates that with increasing value
for $\lambda_1$, the performance skews more and more towards *Learning Shapelets*. By keeping a moderate value of $\lambda_1=0.1$ (indicated by a green check mark) we achieve similar accuracies comapred to *Learning Shapelets*. Later, we will see that a value of $\lambda_1=0.1$ already seems to be enough to produce mostly shapelets with realistic patterns.

#### Adversarially-Regularized Shapelets [9] vs Adapted Learning Shapelets

![shapelet_acc_ar_dist](images/stats/accuracies_adversarial_vs_dist_loss.png)

**Figure 3**: Each subplot shows a comparison of the best run according to accuracy of *Learning Interpretable Shapelets through Adversarial Regularization* [8] and $\mathcal{L_R}$ with varying values for  $\lambda_1$ . The values on the y-axis correspond to the accuracy of *Learning Interpretable Shapelets through Adversarial Regularization* and the values on the x-axis to the accuracy of learing wit $\mathcal{L_R}$. The green check marks indicate that $\mathcal{L_R}$ significantly outperforms *Learning Interpretable Shapelets through Adversarial Regularization*.

## Runtime 

We compare the runtime of *Learning Shapelets* loss to learning with $\mathcal{L_R}$. We can observe that training of *Learning Shapelets* converges quicker than learning with $\mathcal{L_R}$. On average *Learning Shapelets* was six times quicker than learning with $\mathcal{L_R}$ (See Tables 2 and 3 below for the precise numbers). 

![shapelts_gpu_time](images/stats/gputime_all_datasets_by_length.png)

**Figure 4**: GPU time in seconds for increasing dataset size. The size is computed as the total number of time steps $number\_of\_samples \cdot length\_of\_sample$. Note that the x scale is not linear, the total number of time steps varies from 1400 to 2,709,000.  
**Left**: Raw GPU time of each approach.  
**Right**: GPU time of each approach. The curves are smoothed with a gaussian kernel (sigma=1).

### Shapelet Quality

By shapelet quality we refer to the closeness of the shapelet to the actual data. The closer to the data the better the quality. If the learned shapelets are not close to the dataset, interpretability becomes more difficult.

To evaluate if the learned shapelets actually represent more realistic patterns, we
calculated the average distance of each shapelets to its 10 best matching time series. We can see, that the loss has an noticeable effect on the distance of the shapelets to the data. See Figure 1 for an example on how that difference can actually look like. 

Under `experimental_results/images/shapelets` you can have a look on the learned shapelets for each of the 85 datasets. Often, initialization via k-means already leads to relatively good shapelets, but using the regularizer almost guarantees shapelets that well approximate a pattern in the dataset.

![shapelet_dists](images/stats/shapelet_dists.png)

**Figure 5**: The euclidean distances of each shapelet to its 10 best matching time series. The plot shows all shapelets of the best run for each dataset. So, for the 85 datasets around 2000-2500 shapelets have been learned, i.e. on average not more than 30 shapelets per dataset. On the y-axis we see the distances and on the x-axis the shapelets sorted by distance.  
*Left:* On the left we see the raw euclidean distance.  
*Right*: On the right we see the euclidean distances normalized by the length of the shapelets.
We can see a substantial difference between *Learning Shapelets* (the blue line) and learning with $\mathcal{L_R}$. Also note, that choosing a value of $\lambda_1$ in the range between 0.1-0.4 did not make much of difference, we can only see further improvement by a value of 0.8. But a value of 0.8 for $\lambda_1$ only seems to marginally increase the distances of the shapelets to the data and because higher values slowly result in a loss of prediction accuracy (indicated by Figure 2), we recommend setting $\lambda_1$ to the value 0.1. 

## Concluding Remarks

Using $\mathcal{L_R}$ consistently improves the quality of the learned shapelets. A regularization strength of $\lambda_1=0.1$ does not result in a significant loss of accuracy compared to solely using cross-entropy loss. But, the runtime increases on average by around 6 times compared to learning solely with cross-entropy loss. 

Further, the effects of the regularizer $\lambda_2$ has not been evaluated so far.

We added the option to train the regularizer with cosine similarity. This is experimental and it is not recommended to use at the moment. Our preliminary results seem to indicate that it does not lead to similar good results as to use euclidean distance.

See below for tables showing our precise results.

## References

[1] Lexiang Ye and Eamonn Keogh. Time series shapelets: A new primitive for data mining. In Proceedings of the 15th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, KDD ’09, page 947–956, New York, NY, USA, 2009. Association for Computing Machinery.

[2] Thanawin Rakthanmanon and Eamonn Keogh. Fast Shapelets: A Scalable Algorithm for Discovering Time Series Shapelets. In Proceedings of the 2013 SIAM International Conference on Data Mining, pages 668–676. Society for Industrial and Applied Mathematics, May 2013.

[3] Martin Wistuba, Josif Grabocka, and Lars Schmidt-Thieme. Ultra-Fast Shapelets for Time Series Classification. page 22.

[4] Josif Grabocka, Martin Wistuba, and Lars Schmidt-Thieme. Fast classification of univariate and multivariate time series through shapelet discovery. Knowledge and Information Systems, 49(2):429–454, November 2016. Number: 2.

[5] Abdullah Mueen, Eamonn Keogh, and Neal Young. Logical-shapelets: An expressive primitive for time series classification. In Proceedings of the 17th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, KDD ’11, page 1154–1162, New York, NY, USA, 2011. Association for Computing Machinery.

[6] Cun Ji, Chao Zhao, Shijun Liu, Chenglei Yang, Li Pan, Lei Wu, and Xiangxu Meng. A fast shapelet selection algorithm for time series classification. Computer Networks, 148:231–240, 
January 2019.

[7] Josif Grabocka, Nicolas Schilling, Martin Wistuba, and Lars Schmidt-Thieme. Learning time-series shapelets. In Proceedings of the 20th ACM SIGKDD international conference on Knowledge discovery and data mining - KDD ’14, pages 392–401, New York, New York, USA, 2014. ACM Press.

[8] Yichang Wang, Rémi Emonet, Elisa Fromont, Simon Malinowski, Etienne Menager, Loı̈c Mosser, and Romain Tavenard. Learning Interpretable Shapelets for Time Series Classification through Adversarial Regularization. arXiv:1906.00917 [cs, stat], June 2019. arXiv: 1906.00917.

[9] Zicheng Fang, Peng Wang, and Wei Wang. Efficient Learning Interpretable Shapelets for Accurate Time Series Classification. In 2018 IEEE 34th International Conference on Data Engineering (ICDE), pages 497–508, Paris, April 2018. IEEE.

[10] David Guijo-Rubio, Pedro A Gutiérrez, Romain Tavenard, and Anthony Bagnall. A Hybrid Approach to Time Series Classification with Shapelets. In Intelligent Data Engineering and Automated Learning – IDEAL, pages 137–144, Manchester, United Kingdom, November 2019.

[11] Kidger, Patrick et al. “Generalised Interpretable Shapelets for Irregular Time Series.” ArXiv abs/2005.13948 (2020): n. pag.

[12] Hoang Anh Dau, Anthony J. Bagnall, Kaveh Kamgar, Chin-Chia Michael Yeh, Yan Zhu, Shaghayegh Gharghabi, Chotirat Ann Ratanamahatana, and Eamonn J. Keogh. The UCR time series archive. CoRR, abs/1810.07758, 2018.

[13] Kingma, Diederik P., and Jimmy Ba. "Adam: A method for stochastic optimization." arXiv preprint arXiv:1412.6980 (2014).

## Appendix

### Table Accuracies
The table shows the actual accuracies of the proposed loss function and a comparison to some related approaches.

|Dataset | Wang et al. | Ultra Fast Shapelets | Shapelet Transform | Scalable Shapelets | Logical Shapelets | Learning Shapelets (Paper) | Learning Shapelets (Our) | Histogram Loss | Fast Shapelets | Fang et al. | **This (l1=0.8, l2=0.01, k=0.1*batch_size)** | **This (l1=0.4, l2=0.01, k=0.1*batch_size)** | **This (l1=0.2, l2=0.01, k=0.1*batch_size)** | **This (l1=0.1, l2=0.01, k=0.1*batch_size)** |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| Adiac | 0.399 | 0.724 | 0.2379 | 0.583 | 0.5857 | 0.542 | 0.7263 | None | 0.4859 | None | 0.7391 | 0.734 | 0.734 | **0.7494** |
| ArrowHead | 0.8171 | None | None | None | None | None | 0.84 | 0.8229 | None | None | 0.84 | 0.84 | **0.8571** | 0.84 |
| Beef | 0.7667 | 0.737 | **0.8667** | 0.507 | 0.5667 | 0.8 | 0.8333 | 0.8 | 0.5533 | 0.63333 | 0.8 | 0.7667 | 0.8 | 0.8333 |
| BeetleFly | 0.7 | None | None | None | None | 0.95 | 0.95 | **1.0** | None | 0.85 | 0.95 | 0.9 | 0.95 | 0.95 |
| BirdChicken | 0.9 | None | None | None | None | **1.0** | **1.0** | **1.0** | None | 0.9 | **1.0** | **1.0** | **1.0** | **1.0** |
| CBF | 0.9867 | **0.998** | None | 0.975 | 0.8856 | None | 0.9944 | 0.9911 | 0.9471 | None | 0.9922 | 0.9956 | 0.9911 | 0.9922 |
| Car | 0.75 | None | None | None | None | None | **0.9167** | 0.8833 | None | None | 0.8667 | 0.85 | 0.8833 | 0.85 |
| ChlorineConcentration | 0.6596 | 0.738 | 0.5615 | 0.553 | 0.6185 | **0.743** | 0.6818 | 0.675 | 0.5831 | None | 0.6128 | 0.6346 | 0.6318 | 0.6484 |
| CinCECGTorso | 0.7341 | **0.849** | None | 0.773 | None | None | 0.8196 | None | 0.6986 | None | 0.7703 | 0.7739 | 0.7899 | 0.7841 |
| Coffee | **1.0** | 0.989 | **1.0** | 0.961 | 0.9643 | **1.0** | **1.0** | **1.0** | 0.9321 | 0.9642 | **1.0** | **1.0** | **1.0** | **1.0** |
| Computers | 0.58 | None | None | None | None | None | **0.74** | 0.704 | None | None | 0.704 | 0.708 | 0.72 | 0.728 |
| CricketX | 0.7513 | **0.775** | None | 0.672 | None | None | 0.7077 | 0.7205 | None | None | 0.6385 | 0.6718 | 0.6872 | 0.7103 |
| CricketY | 0.7205 | **0.766** | None | 0.675 | None | None | 0.6923 | 0.7359 | None | None | 0.6385 | 0.6667 | 0.6564 | 0.6923 |
| CricketZ | 0.7564 | **0.793** | None | 0.673 | None | None | 0.7077 | 0.7385 | None | None | 0.641 | 0.6641 | 0.6923 | 0.7128 |
| DiatomSizeReduction | 0.9804 | 0.925 | 0.9216 | 0.896 | 0.8007 | 0.951 | 0.9706 | 0.9869 | 0.883 | 0.8986 | 0.9739 | **0.9935** | 0.9869 | 0.9837 |
| DistalPhalanxOutlineAgeGroup | 0.7266 | None | None | None | None | None | 0.7626 | **0.777** | None | None | 0.7626 | **0.777** | 0.7698 | 0.7626 |
| DistalPhalanxOutlineCorrect | 0.7464 | None | None | None | None | None | 0.7754 | **0.808** | None | None | 0.7681 | 0.7754 | 0.7899 | 0.8007 |
| DistalPhalanxTW | 0.6691 | None | None | None | None | None | 0.7122 | **0.7266** | None | None | 0.705 | 0.6906 | 0.705 | 0.705 |
| ECG200 | 0.92 | 0.881 | None | 0.818 | 0.87 | None | 0.9 | 0.9 | 0.773 | None | 0.87 | 0.88 | 0.9 | **0.94** |
| ECG5000 | 0.9287 | None | None | None | None | None | **0.944** | 0.9376 | None | None | 0.9404 | 0.942 | 0.9389 | 0.94 |
| ECGFiveDays | 0.9977 | **1.0** | 0.9895 | 0.953 | 0.9942 | **1.0** | **1.0** | **1.0** | 0.9959 | **1.0** | **1.0** | **1.0** | **1.0** | **1.0** |
| Earthquakes | 0.6763 | None | None | None | None | None | **0.7626** | 0.7554 | None | None | 0.7554 | 0.7554 | **0.7626** | 0.7554 |
| ElectricDevices | 0.5346 | None | 0.2425 | None | None | None | **0.7249** | 0.693 | None | None | 0.6925 | 0.692 | 0.6903 | 0.6955 |
| FaceAll | 0.7568 | 0.788 | None | 0.714 | 0.6586 | None | **0.8101** | None | 0.5893 | None | **0.8101** | 0.7852 | 0.7828 | 0.8012 |
| FaceFour | 0.8295 | 0.961 | 0.9773 | 0.82 | 0.4886 | **1.0** | 0.9659 | 0.9886 | 0.9102 | 0.9545 | 0.9545 | 0.9659 | 0.9545 | 0.9773 |
| FacesUCR | 0.8566 | **0.957** | None | 0.847 | 0.6624 | None | 0.9254 | 0.9176 | 0.6717 | None | 0.8498 | 0.8805 | 0.902 | 0.899 |
| FiftyWords | 0.7077 | **0.816** | None | 0.68 | None | None | 0.7692 | 0.7473 | None | None | 0.7275 | 0.7319 | 0.7231 | 0.7407 |
| Fish | 0.8171 | **0.965** | None | 0.755 | 0.7771 | None | 0.96 | 0.9486 | 0.8028 | None | 0.9314 | 0.9429 | 0.9429 | 0.9371 |
| FordA | 0.8803 | None | None | None | None | None | 0.9227 | **0.925** | None | None | 0.8705 | 0.8841 | 0.9083 | 0.9083 |
| FordB | 0.7765 | None | None | None | None | None | 0.8012 | **0.8086** | None | None | 0.7642 | 0.7728 | 0.7877 | 0.8025 |
| GunPoint | 0.9733 | 0.979 | **1.0** | 0.931 | 0.8933 | **1.0** | **1.0** | **1.0** | 0.9393 | 0.9933 | **1.0** | **1.0** | **1.0** | **1.0** |
| Ham | 0.7048 | None | None | None | None | None | **0.819** | 0.8 | None | None | 0.7905 | 0.7714 | 0.7905 | 0.781 |
| HandOutlines | 0.8973 | None | None | None | None | None | **0.9351** | None | None | None | 0.9189 | 0.9216 | 0.927 | 0.9324 |
| Haptics | 0.4091 | **0.491** | None | 0.356 | None | None | 0.4838 | None | None | None | 0.461 | 0.4578 | 0.4708 | 0.4708 |
| Herring | 0.5625 | None | None | None | None | None | **0.7188** | **0.7188** | None | None | **0.7188** | **0.7188** | 0.6875 | **0.7188** |
| InlineSkate | 0.3764 | 0.382 | None | 0.385 | None | None | 0.4073 | None | None | None | 0.3855 | 0.4236 | 0.4073 | **0.4273** |
| InsectWingbeatSound | **0.6051** | None | None | None | None | None | 0.5975 | 0.6025 | None | None | 0.5576 | 0.5631 | 0.5601 | 0.597 |
| ItalyPowerDemand | 0.9514 | 0.962 | 0.9213 | 0.92 | 0.9359 | 0.962 | 0.9563 | None | 0.9051 | **0.9757** | 0.9485 | 0.9349 | 0.932 | 0.9563 |
| LargeKitchenAppliances | 0.624 | None | None | None | None | None | **0.8933** | 0.8907 | None | None | 0.864 | 0.8773 | 0.8747 | 0.8747 |
| Lightning2 | 0.8033 | 0.787 | None | 0.795 | 0.4262 | None | 0.8689 | **0.8852** | 0.7049 | None | **0.8852** | 0.8525 | **0.8852** | 0.8525 |
| Lightning7 | 0.8356 | 0.778 | 0.6986 | 0.652 | 0.5479 | **0.877** | 0.7808 | 0.8356 | 0.5973 | 0.8082 | 0.8219 | 0.8219 | 0.8082 | 0.863 |
| Mallat | 0.9561 | 0.956 | None | 0.926 | 0.6563 | None | 0.9505 | None | 0.9672 | None | **0.991** | 0.9804 | 0.9821 | 0.9723 |
| Meat | 0.8667 | None | None | None | None | None | 0.9833 | **1.0** | None | None | 0.9833 | **1.0** | **1.0** | **1.0** |
| MedicalImages | 0.6895 | 0.74 | 0.525 | 0.676 | 0.5868 | 0.734 | **0.7474** | 0.6737 | 0.567 | None | 0.6737 | 0.6882 | 0.7039 | 0.7066 |
| MiddlePhalanxOutlineAgeGroup | 0.6039 | None | None | None | None | None | 0.6364 | 0.6494 | None | None | 0.6429 | 0.6429 | 0.6494 | **0.6558** |
| MiddlePhalanxOutlineCorrect | 0.7732 | None | None | None | None | None | **0.8316** | 0.811 | None | None | 0.8041 | 0.8041 | 0.8213 | 0.8076 |
| MiddlePhalanxTW | 0.513 | None | None | None | None | None | **0.6299** | 0.6234 | None | None | 0.6234 | 0.6234 | 0.6169 | **0.6299** |
| MoteStrain | 0.8395 | 0.866 | 0.8866 | 0.783 | 0.8323 | 0.913 | **0.9137** | 0.881 | 0.7828 | 0.8977 | 0.8706 | 0.8722 | 0.8746 | 0.8946 |
| NonInvasiveFetalECGThorax1 | 0.8137 | None | None | 0.814 | None | None | 0.915 | None | None | None | 0.8936 | 0.9079 | 0.9155 | **0.917** |
| NonInvasiveFetalECGThorax2 | 0.8656 | None | None | 0.855 | None | None | 0.9298 | None | None | None | 0.9211 | 0.9247 | 0.9267 | **0.9344** |
| OSULeaf | 0.6322 | **0.842** | None | 0.566 | 0.686 | None | 0.8017 | 0.7893 | 0.6409 | None | 0.7645 | 0.7769 | 0.7934 | 0.781 |
| OliveOil | 0.7667 | 0.89 | None | 0.79 | 0.8333 | None | 0.8667 | **0.9** | 0.7867 | None | 0.8667 | 0.8667 | 0.8667 | **0.9** |
| PhalangesOutlinesCorrect | 0.7751 | None | None | None | None | None | 0.7681 | None | None | None | 0.7727 | 0.7879 | 0.7879 | **0.7972** |
| Phoneme | 0.1772 | None | None | None | None | None | **0.2025** | None | None | None | 0.1693 | 0.1735 | 0.1799 | 0.1825 |
| Plane | 0.9524 | None | None | None | None | None | **1.0** | **1.0** | None | None | **1.0** | **1.0** | **1.0** | **1.0** |
| ProximalPhalanxOutlineAgeGroup | 0.7951 | None | None | None | None | None | **0.8634** | 0.8585 | None | None | 0.8585 | **0.8634** | 0.8585 | **0.8634** |
| ProximalPhalanxOutlineCorrect | 0.8076 | None | None | None | None | None | 0.8729 | 0.8729 | None | None | 0.8557 | 0.866 | **0.89** | 0.8763 |
| ProximalPhalanxTW | 0.7268 | None | None | None | None | None | 0.8195 | 0.8146 | None | None | 0.8195 | **0.8293** | 0.8195 | 0.8146 |
| RefrigerationDevices | 0.5067 | None | None | None | None | None | **0.6107** | 0.584 | None | None | 0.6 | 0.592 | 0.608 | 0.5947 |
| ScreenType | 0.368 | None | None | None | None | None | 0.496 | **0.5493** | None | None | 0.4747 | 0.4693 | 0.4853 | 0.5307 |
| ShapeletSim | 0.6 | None | None | None | None | None | **1.0** | **1.0** | None | None | **1.0** | **1.0** | **1.0** | **1.0** |
| ShapesAll | 0.7933 | None | None | None | None | None | 0.8217 | None | None | None | 0.8033 | 0.83 | 0.8233 | **0.8333** |
| SmallKitchenAppliances | 0.5173 | None | None | None | None | None | 0.736 | 0.736 | None | None | 0.7253 | 0.768 | **0.7733** | 0.7493 |
| SonyAIBORobotSurface1 | 0.7388 | 0.878 | 0.8669 | 0.85 | 0.8602 | 0.952 | 0.9584 | **0.97** | 0.6855 | 0.8785 | 0.9135 | 0.9251 | 0.9368 | 0.9268 |
| SonyAIBORobotSurface2 | 0.7996 | 0.912 | None | 0.78 | 0.8458 | None | 0.915 | **0.9255** | 0.7852 | None | 0.9056 | 0.9129 | 0.9234 | 0.9161 |
| StarLightCurves | 0.9655 | **0.969** | None | 0.933 | None | None | 0.9592 | None | None | None | 0.9495 | 0.9526 | 0.9474 | 0.9542 |
| Strawberry | 0.927 | None | None | None | None | None | 0.9595 | 0.9568 | None | None | 0.9568 | 0.9649 | 0.9676 | **0.9703** |
| SwedishLeaf | 0.8528 | **0.935** | None | 0.849 | 0.8128 | None | 0.9232 | 0.92 | 0.7307 | None | 0.9104 | 0.9104 | 0.9296 | 0.9184 |
| Symbols | 0.8422 | 0.889 | 0.8462 | 0.865 | 0.6432 | **1.0** | 0.9497 | 0.9518 | 0.9324 | 0.7829 | 0.9508 | 0.9528 | 0.9558 | 0.9528 |
| SyntheticControl | 0.9733 | 0.994 | 0.8733 | 0.983 | 0.47 | **1.0** | 0.9867 | 0.9867 | 0.919 | None | 0.9833 | 0.99 | 0.99 | 0.9867 |
| ToeSegmentation1 | 0.8816 | None | None | None | None | None | **0.9649** | **0.9649** | None | None | 0.9342 | 0.9561 | 0.9518 | 0.9561 |
| ToeSegmentation2 | 0.8462 | None | None | None | None | None | **0.9308** | **0.9308** | None | None | 0.9 | **0.9308** | 0.9154 | 0.9231 |
| Trace | 0.99 | **1.0** | 0.98 | 0.965 | **1.0** | **1.0** | **1.0** | **1.0** | 0.998 | **1.0** | **1.0** | **1.0** | **1.0** | **1.0** |
| TwoLeadECG | 0.9385 | 0.975 | 0.993 | 0.867 | 0.856 | **1.0** | 0.9965 | 0.9991 | 0.9097 | 0.9982 | 0.9868 | 0.9868 | 0.9903 | 0.9947 |
| TwoPatterns | 0.991 | **0.997** | None | 0.981 | 0.539 | None | 0.981 | 0.9912 | 0.8865 | None | 0.9508 | 0.9578 | 0.9732 | 0.9782 |
| UWaveGestureLibraryAll | **0.9531** | None | None | None | None | None | 0.9165 | 0.9369 | None | None | 0.9213 | 0.9232 | 0.9288 | 0.9341 |
| UWaveGestureLibraryX | 0.7786 | **0.803** | None | 0.761 | None | None | 0.7669 | 0.7842 | None | None | 0.7722 | 0.7697 | 0.775 | 0.7719 |
| UWaveGestureLibraryY | 0.6943 | **0.709** | None | 0.671 | None | None | 0.6803 | 0.6924 | None | None | 0.6812 | 0.6792 | 0.6745 | 0.6879 |
| UWaveGestureLibraryZ | 0.696 | **0.729** | None | 0.676 | None | None | 0.7111 | 0.7228 | None | None | 0.7113 | 0.7219 | 0.727 | 0.7194 |
| Wafer | 0.9935 | 0.997 | None | 0.993 | **0.9988** | None | 0.9958 | 0.9951 | 0.9964 | None | 0.9911 | 0.9911 | 0.9921 | 0.9922 |
| Wine | 0.7037 | None | None | None | None | None | 0.9815 | 0.9259 | None | None | 0.9444 | 0.9259 | **1.0** | 0.963 |
| WordSynonyms | 0.6082 | **0.731** | None | 0.625 | None | None | 0.6646 | None | None | None | 0.594 | 0.6144 | 0.6505 | 0.6442 |
| Worms | 0.5325 | None | None | None | None | None | 0.6753 | 0.6753 | None | None | 0.6494 | 0.6883 | 0.6364 | **0.7143** |
| WormsTwoClass | 0.7013 | None | None | None | None | None | **0.8052** | **0.8052** | None | None | 0.7922 | 0.7922 | **0.8052** | **0.8052** |
| Yoga | 0.8133 | **0.871** | None | 0.625 | 0.74 | None | 0.856 | 0.859 | 0.7507 | None | 0.822 | 0.825 | 0.8483 | 0.856 |
|Wins| 3 | 20 | 3 | 0 | 2 | 11 | 27 | 23 | 0 | 3 | 11 | 14 | 15 | 23 |


Table 1.

## Table total GPU training time in seconds

The table shows the runtime in **seconds** of the best run respectively.

| Dataset | Grabocka et al. | **This (l1=0.1, l2=0.01)** | **This (l1=0.2, l2=0.01)** | **This (l1=0.4, l2=0.01)** | **This (l1=0.8, l2=0.01)** |
|-------|-------|-------|-------|-------|-------|
| Car | **3.7** | 20.17 | 13.51 | 13.98 | 19.76 |
| ArrowHead | **2.45** | 10.02 | 8.8 | 13.68 | 18.46 |
| Adiac | **40.46** | 300.9 | 270.8 | 226.08 | 265.34 |
| BirdChicken | **1.49** | 4.74 | 7.56 | 7.56 | 9.18 |
| Beef | **7.3** | 23.26 | 9.87 | 13.0 | 13.41 |
| ChlorineConcentration | **31.54** | 360.16 | 251.23 | 281.56 | 260.24 |
| BeetleFly | **2.43** | 14.44 | 5.32 | 6.57 | 8.36 |
| CBF | **2.47** | 14.31 | 9.27 | 12.88 | 12.68 |
| CinCECGTorso | **6.01** | 28.62 | 26.67 | 26.24 | 27.08 |
| Computers | 39.14 | 25.85 | **25.0** | 37.28 | 37.76 |
| Coffee | **2.69** | 4.31 | 5.88 | 7.74 | 5.38 |
| CricketX | **15.03** | 146.64 | 132.83 | 152.24 | 110.35 |
| CricketY | **16.56** | 126.53 | 152.85 | 144.01 | 205.51 |
| CricketZ | **17.6** | 121.72 | 137.81 | 218.45 | 189.31 |
| DiatomSizeReduction | **2.73** | 9.85 | 6.9 | 7.27 | 13.4 |
| DistalPhalanxOutlineAgeGroup | **20.44** | 120.8 | 45.76 | 68.51 | 129.81 |
| DistalPhalanxOutlineCorrect | **14.98** | 178.39 | 64.6 | 129.73 | 99.17 |
| DistalPhalanxTW | **17.49** | 169.85 | 79.8 | 89.03 | 52.87 |
| ECG200 | **8.01** | 25.44 | 24.79 | 34.83 | 31.2 |
| ECG5000 | **13.26** | 130.66 | 46.64 | 127.07 | 47.45 |
| ECGFiveDays | **1.98** | 5.66 | 7.87 | 7.88 | 6.95 |
| ElectricDevices | **154.41** | 1195.95 | 1061.52 | 1050.65 | 1412.83 |
| Earthquakes | **14.79** | 59.56 | 34.94 | 40.0 | 40.95 |
| FaceFour | **1.55** | 6.35 | 7.28 | 7.61 | 11.46 |
| FaceAll | **20.67** | 190.62 | 190.21 | 153.53 | 161.56 |
| FacesUCR | **4.91** | 31.13 | 31.36 | 30.77 | 41.56 |
| FiftyWords | **12.23** | 109.69 | 94.04 | 94.5 | 106.8 |
| Fish | **8.94** | 45.15 | 57.35 | 43.48 | 49.99 |
| FordA | **44.38** | 1175.45 | 617.32 | 813.38 | 710.1 |
| FordB | **35.77** | 923.21 | 830.72 | 824.86 | 369.16 |
| GunPoint | **4.01** | 11.43 | 12.39 | 12.37 | 14.27 |
| Ham | **3.81** | 33.75 | 14.54 | 29.87 | 20.63 |
| HandOutlines | **849.33** | 919.75 | 885.99 | 906.66 | 991.64 |
| Haptics | **33.39** | 179.56 | 179.63 | 49.89 | 254.92 |
| Herring | **4.27** | 10.26 | 31.33 | 34.38 | 18.52 |
| InlineSkate | **40.9** | 61.6 | 138.39 | 98.73 | 111.4 |
| InsectWingbeatSound | **8.53** | 91.06 | 102.45 | 87.66 | 85.55 |
| ItalyPowerDemand | **30.68** | 39.79 | 40.87 | 41.79 | 40.59 |
| LargeKitchenAppliances | **33.64** | 74.96 | 89.88 | 90.42 | 116.43 |
| Lightning2 | **2.61** | 8.27 | 16.35 | 15.93 | 18.11 |
| MiddlePhalanxOutlineCorrect | **33.67** | 172.02 | 137.27 | 176.3 | 150.95 |
| Lightning7 | **6.54** | 21.74 | 23.76 | 23.1 | 25.12 |
| Mallat | **2.83** | 28.0 | 26.08 | 25.74 | 37.7 |
| Meat | **2.94** | 5.66 | 6.03 | 9.1 | 13.25 |
| MiddlePhalanxTW | **9.6** | 42.65 | 46.3 | 22.77 | 104.43 |
| MedicalImages | **26.66** | 78.96 | 154.38 | 148.65 | 126.64 |
| NonInvasiveFetalECGThorax1 | **130.01** | 611.05 | 1040.62 | 834.55 | 520.51 |
| MiddlePhalanxOutlineAgeGroup | **9.75** | 82.26 | 61.5 | 60.23 | 79.54 |
| MoteStrain | **1.72** | 4.28 | 6.69 | 6.91 | 4.52 |
| OliveOil | **3.96** | 34.87 | 20.2 | 27.92 | 34.48 |
| NonInvasiveFetalECGThorax2 | **161.88** | 1022.59 | 940.74 | 931.94 | 625.74 |
| OSULeaf | **6.84** | 30.28 | 30.99 | 30.98 | 51.97 |
| PhalangesOutlinesCorrect | **56.96** | 206.08 | 77.61 | 75.97 | 188.69 |
| Plane | **5.84** | 14.07 | 8.79 | 9.94 | 26.89 |
| Phoneme | **14.47** | 280.84 | 294.73 | 283.17 | 411.27 |
| ProximalPhalanxOutlineAgeGroup | **13.3** | 128.69 | 131.77 | 131.76 | 92.54 |
| SmallKitchenAppliances | **53.11** | 181.64 | 91.79 | 72.23 | 60.08 |
| ProximalPhalanxOutlineCorrect | **17.79** | 110.48 | 114.13 | 169.49 | 163.63 |
| ProximalPhalanxTW | **12.05** | 66.81 | 135.0 | 209.88 | 118.02 |
| RefrigerationDevices | **9.89** | 53.06 | 25.31 | 33.61 | 54.44 |
| SonyAIBORobotSurface2 | **1.56** | 7.18 | 5.68 | 6.93 | 7.08 |
| ScreenType | **18.46** | 116.52 | 164.33 | 75.9 | 70.19 |
| ShapeletSim | **1.49** | 4.05 | 4.26 | 4.18 | 4.03 |
| ShapesAll | **18.95** | 132.28 | 94.56 | 123.24 | 154.56 |
| SwedishLeaf | **15.53** | 183.34 | 183.81 | 179.46 | 157.24 |
| SonyAIBORobotSurface1 | **1.56** | 4.5 | 9.61 | 6.9 | 7.35 |
| StarLightCurves | **31.68** | 529.05 | 1001.4 | 1071.07 | 532.44 |
| Strawberry | **21.9** | 186.13 | 201.47 | 182.0 | 259.77 |
| Symbols | **2.43** | 10.05 | 9.56 | 15.07 | 9.1 |
| SyntheticControl | **18.01** | 36.34 | 56.55 | 39.38 | 38.05 |
| ToeSegmentation1 | **3.67** | 14.11 | 10.21 | 14.25 | 15.05 |
| ToeSegmentation2 | **2.17** | 13.87 | 7.65 | 7.84 | 13.06 |
| Trace | **3.87** | 8.99 | 13.77 | 14.47 | 13.77 |
| UWaveGestureLibraryZ | **34.96** | 202.3 | 164.36 | 291.5 | 208.82 |
| TwoLeadECG | **1.66** | 4.51 | 6.69 | 4.55 | 4.64 |
| TwoPatterns | **13.28** | 124.62 | 168.69 | 212.51 | 253.19 |
| Wafer | **11.55** | 122.34 | 130.88 | 129.66 | 196.72 |
| UWaveGestureLibraryAll | **95.01** | 283.12 | 279.81 | 421.97 | 420.61 |
| UWaveGestureLibraryX | **29.73** | 213.8 | 111.45 | 244.37 | 263.55 |
| Worms | **14.06** | 77.08 | 43.01 | 89.8 | 155.72 |
| UWaveGestureLibraryY | **37.53** | 202.58 | 141.05 | 163.11 | 163.48 |
| Wine | **7.79** | 20.93 | 31.16 | 44.92 | 24.48 |
| WordSynonyms | **11.28** | 88.59 | 130.48 | 106.47 | 87.67 |
| WormsTwoClass | **17.72** | 49.55 | 58.9 | 99.18 | 25.45 |
| Yoga | **16.45** | 149.53 | 72.97 | 92.0 | 189.47 |
|Wins| 84 | 0 | 1 | 0 | 0 |


Table 2.

## Relative GPU time of $\mathcal{L_R}$

The table shows the training time in multiples of the training time of *Learning Shapelets*. Training with $\mathcal{L_R}$ is on average around 6 times slower.

| Dataset | Grabocka et al. (time in sec.) | Grabocka et al. (reference time) | **This (l1=0.1, l2=0.01)** | **This (l1=0.2, l2=0.01)** | **This (l1=0.4, l2=0.01)** | **This (l1=0.8, l2=0.01)** |
|-------|-------|-------|-------|-------|-------|-------|
| Car | 3.7 | **1** | 5.45 | 3.65 | 3.78 | 5.34 |
| ArrowHead | 2.45 | **1** | 4.09 | 3.59 | 5.58 | 7.53 |
| Adiac | 40.46 | **1** | 7.44 | 6.69 | 5.59 | 6.56 |
| BirdChicken | 1.49 | **1** | 3.18 | 5.07 | 5.07 | 6.16 |
| Beef | 7.3 | **1** | 3.19 | 1.35 | 1.78 | 1.84 |
| ChlorineConcentration | 31.54 | **1** | 11.42 | 7.97 | 8.93 | 8.25 |
| BeetleFly | 2.43 | **1** | 5.94 | 2.19 | 2.7 | 3.44 |
| CBF | 2.47 | **1** | 5.79 | 3.75 | 5.21 | 5.13 |
| CinCECGTorso | 6.01 | **1** | 4.76 | 4.44 | 4.37 | 4.51 |
| Computers | 39.14 | 1 | 0.66 | **0.64** | 0.95 | 0.96 |
| Coffee | 2.69 | **1** | 1.6 | 2.19 | 2.88 | 2.0 |
| CricketX | 15.03 | **1** | 9.76 | 8.84 | 10.13 | 7.34 |
| CricketY | 16.56 | **1** | 7.64 | 9.23 | 8.7 | 12.41 |
| CricketZ | 17.6 | **1** | 6.92 | 7.83 | 12.41 | 10.76 |
| DiatomSizeReduction | 2.73 | **1** | 3.61 | 2.53 | 2.66 | 4.91 |
| DistalPhalanxOutlineAgeGroup | 20.44 | **1** | 5.91 | 2.24 | 3.35 | 6.35 |
| DistalPhalanxOutlineCorrect | 14.98 | **1** | 11.91 | 4.31 | 8.66 | 6.62 |
| DistalPhalanxTW | 17.49 | **1** | 9.71 | 4.56 | 5.09 | 3.02 |
| ECG200 | 8.01 | **1** | 3.18 | 3.09 | 4.35 | 3.9 |
| ECG5000 | 13.26 | **1** | 9.85 | 3.52 | 9.58 | 3.58 |
| ECGFiveDays | 1.98 | **1** | 2.86 | 3.97 | 3.98 | 3.51 |
| ElectricDevices | 154.41 | **1** | 7.75 | 6.87 | 6.8 | 9.15 |
| Earthquakes | 14.79 | **1** | 4.03 | 2.36 | 2.7 | 2.77 |
| FaceFour | 1.55 | **1** | 4.1 | 4.7 | 4.91 | 7.39 |
| FaceAll | 20.67 | **1** | 9.22 | 9.2 | 7.43 | 7.82 |
| FacesUCR | 4.91 | **1** | 6.34 | 6.39 | 6.27 | 8.46 |
| FiftyWords | 12.23 | **1** | 8.97 | 7.69 | 7.73 | 8.73 |
| Fish | 8.94 | **1** | 5.05 | 6.41 | 4.86 | 5.59 |
| FordA | 44.38 | **1** | 26.49 | 13.91 | 18.33 | 16.0 |
| FordB | 35.77 | **1** | 25.81 | 23.22 | 23.06 | 10.32 |
| GunPoint | 4.01 | **1** | 2.85 | 3.09 | 3.08 | 3.56 |
| Ham | 3.81 | **1** | 8.86 | 3.82 | 7.84 | 5.41 |
| HandOutlines | 849.33 | **1** | 1.08 | 1.04 | 1.07 | 1.17 |
| Haptics | 33.39 | **1** | 5.38 | 5.38 | 1.49 | 7.63 |
| Herring | 4.27 | **1** | 2.4 | 7.34 | 8.05 | 4.34 |
| InlineSkate | 40.9 | **1** | 1.51 | 3.38 | 2.41 | 2.72 |
| InsectWingbeatSound | 8.53 | **1** | 10.68 | 12.01 | 10.28 | 10.03 |
| ItalyPowerDemand | 30.68 | **1** | 1.3 | 1.33 | 1.36 | 1.32 |
| LargeKitchenAppliances | 33.64 | **1** | 2.23 | 2.67 | 2.69 | 3.46 |
| Lightning2 | 2.61 | **1** | 3.17 | 6.26 | 6.1 | 6.94 |
| MiddlePhalanxOutlineCorrect | 33.67 | **1** | 5.11 | 4.08 | 5.24 | 4.48 |
| Lightning7 | 6.54 | **1** | 3.32 | 3.63 | 3.53 | 3.84 |
| Mallat | 2.83 | **1** | 9.89 | 9.22 | 9.1 | 13.32 |
| Meat | 2.94 | **1** | 1.93 | 2.05 | 3.1 | 4.51 |
| MiddlePhalanxTW | 9.6 | **1** | 4.44 | 4.82 | 2.37 | 10.88 |
| MedicalImages | 26.66 | **1** | 2.96 | 5.79 | 5.58 | 4.75 |
| NonInvasiveFetalECGThorax1 | 130.01 | **1** | 4.7 | 8.0 | 6.42 | 4.0 |
| MiddlePhalanxOutlineAgeGroup | 9.75 | **1** | 8.44 | 6.31 | 6.18 | 8.16 |
| MoteStrain | 1.72 | **1** | 2.49 | 3.89 | 4.02 | 2.63 |
| OliveOil | 3.96 | **1** | 8.81 | 5.1 | 7.05 | 8.71 |
| NonInvasiveFetalECGThorax2 | 161.88 | **1** | 6.32 | 5.81 | 5.76 | 3.87 |
| OSULeaf | 6.84 | **1** | 4.43 | 4.53 | 4.53 | 7.6 |
| PhalangesOutlinesCorrect | 56.96 | **1** | 3.62 | 1.36 | 1.33 | 3.31 |
| Plane | 5.84 | **1** | 2.41 | 1.51 | 1.7 | 4.6 |
| Phoneme | 14.47 | **1** | 19.41 | 20.37 | 19.57 | 28.42 |
| ProximalPhalanxOutlineAgeGroup | 13.3 | **1** | 9.68 | 9.91 | 9.91 | 6.96 |
| SmallKitchenAppliances | 53.11 | **1** | 3.42 | 1.73 | 1.36 | 1.13 |
| ProximalPhalanxOutlineCorrect | 17.79 | **1** | 6.21 | 6.42 | 9.53 | 9.2 |
| ProximalPhalanxTW | 12.05 | **1** | 5.54 | 11.2 | 17.42 | 9.79 |
| RefrigerationDevices | 9.89 | **1** | 5.37 | 2.56 | 3.4 | 5.5 |
| SonyAIBORobotSurface2 | 1.56 | **1** | 4.6 | 3.64 | 4.44 | 4.54 |
| ScreenType | 18.46 | **1** | 6.31 | 8.9 | 4.11 | 3.8 |
| ShapeletSim | 1.49 | **1** | 2.72 | 2.86 | 2.81 | 2.7 |
| ShapesAll | 18.95 | **1** | 6.98 | 4.99 | 6.5 | 8.16 |
| SwedishLeaf | 15.53 | **1** | 11.81 | 11.84 | 11.56 | 10.12 |
| SonyAIBORobotSurface1 | 1.56 | **1** | 2.88 | 6.16 | 4.42 | 4.71 |
| StarLightCurves | 31.68 | **1** | 16.7 | 31.61 | 33.81 | 16.81 |
| Strawberry | 21.9 | **1** | 8.5 | 9.2 | 8.31 | 11.86 |
| Symbols | 2.43 | **1** | 4.14 | 3.93 | 6.2 | 3.74 |
| SyntheticControl | 18.01 | **1** | 2.02 | 3.14 | 2.19 | 2.11 |
| ToeSegmentation1 | 3.67 | **1** | 3.84 | 2.78 | 3.88 | 4.1 |
| ToeSegmentation2 | 2.17 | **1** | 6.39 | 3.53 | 3.61 | 6.02 |
| Trace | 3.87 | **1** | 2.32 | 3.56 | 3.74 | 3.56 |
| UWaveGestureLibraryZ | 34.96 | **1** | 5.79 | 4.7 | 8.34 | 5.97 |
| TwoLeadECG | 1.66 | **1** | 2.72 | 4.03 | 2.74 | 2.8 |
| TwoPatterns | 13.28 | **1** | 9.38 | 12.7 | 16.0 | 19.07 |
| Wafer | 11.55 | **1** | 10.59 | 11.33 | 11.23 | 17.03 |
| UWaveGestureLibraryAll | 95.01 | **1** | 2.98 | 2.95 | 4.44 | 4.43 |
| UWaveGestureLibraryX | 29.73 | **1** | 7.19 | 3.75 | 8.22 | 8.86 |
| Worms | 14.06 | **1** | 5.48 | 3.06 | 6.39 | 11.08 |
| UWaveGestureLibraryY | 37.53 | **1** | 5.4 | 3.76 | 4.35 | 4.36 |
| Wine | 7.79 | **1** | 2.69 | 4.0 | 5.77 | 3.14 |
| WordSynonyms | 11.28 | **1** | 7.85 | 11.57 | 9.44 | 7.77 |
| WormsTwoClass | 17.72 | **1** | 2.8 | 3.32 | 5.6 | 1.44 |
| Yoga | 16.45 | **1** | 9.09 | 4.44 | 5.59 | 11.52 |
| Average | | 1 | 6.26 | 5.91 | 6.51 | 6.62 |
| Wins | | | 84 | 0 | 1 | 0 | 0 |


Table 3.

## Table Shapelets Configurations

The table shows the shapelets configuration for the best run respectively. For example, `{100: 4}` means: 4 shapelets of length 100.

| Dataset | Grabocka et al. | **This (l1=0.1, l2=0.01)** | **This (l1=0.2, l2=0.01)** | **This (l1=0.4, l2=0.01)** | **This (l1=0.8, l2=0.01)** |
|-------|-------|-------|-------|-------|-------|
| Adiac | {35: 16, 70: 16, 105: 16} | {35: 16, 70: 16, 105: 16} | {35: 16, 70: 16, 105: 16} | {35: 16, 70: 16, 105: 16} | {35: 16, 70: 16, 105: 16} |
| ArrowHead | {50: 8, 100: 8, 150: 8} | {50: 8, 100: 8, 150: 8} | {50: 16} | {50: 8, 100: 8, 150: 8} | {50: 8, 100: 8, 150: 8} |
| Beef | {14: 4, 28: 4, 42: 4} | {14: 16, 28: 16, 42: 16} | {14: 8, 28: 8} | {14: 8, 28: 8} | {14: 8, 28: 8} |
| BeetleFly | {102: 4, 204: 4, 306: 4} | {15: 16, 30: 16, 45: 16} | {102: 16} | {15: 4, 30: 4, 45: 4} | {15: 4, 30: 4, 45: 4} |
| BirdChicken | {102: 4} | {102: 8} | {102: 4, 204: 4} | {102: 4} | {102: 4} |
| CBF | {3: 16, 6: 16, 9: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} |
| Car | {115: 8, 230: 8, 345: 8} | {115: 4, 230: 4} | {17: 16, 34: 16, 51: 16} | {115: 8} | {115: 8} |
| ChlorineConcentration | {33: 16, 66: 16} | {4: 16, 8: 16, 12: 16} | {4: 16, 8: 16, 12: 16} | {4: 16, 8: 16, 12: 16} | {4: 16, 8: 16, 12: 16} |
| CinCECGTorso | {327: 4, 654: 4, 981: 4} | {327: 16, 654: 16, 981: 16} | {327: 16, 654: 16, 981: 16} | {327: 16, 654: 16, 981: 16} | {327: 16, 654: 16, 981: 16} |
| Coffee | {8: 4, 16: 4, 24: 4} | {8: 4} | {8: 4} | {57: 4} | {57: 4} |
| Computers | {21: 4, 42: 4, 63: 4} | {21: 8} | {21: 4, 42: 4, 63: 4} | {21: 8, 42: 8, 63: 8} | {21: 8, 42: 8, 63: 8} |
| CricketX | {60: 16, 120: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} |
| CricketY | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} |
| CricketZ | {60: 16, 120: 16} | {60: 16, 120: 16, 180: 16} | {60: 16, 120: 16, 180: 16} | {60: 8, 120: 8, 180: 8} | {60: 8, 120: 8, 180: 8} |
| DiatomSizeReduction | {69: 4, 138: 4, 207: 4} | {69: 8, 138: 8, 207: 8} | {69: 8, 138: 8} | {69: 4, 138: 4, 207: 4} | {69: 4, 138: 4, 207: 4} |
| DistalPhalanxOutlineAgeGroup | {2: 8, 4: 8, 6: 8} | {2: 8} | {16: 8} | {16: 8} | {16: 8} |
| DistalPhalanxOutlineCorrect | {16: 4, 32: 4, 48: 4} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} |
| DistalPhalanxTW | {16: 8, 32: 8} | {16: 4, 32: 4, 48: 4} | {16: 8} | {16: 4, 32: 4} | {16: 4, 32: 4} |
| ECG200 | {19: 4, 38: 4, 57: 4} | {19: 8, 38: 8, 57: 8} | {19: 8, 38: 8, 57: 8} | {19: 4, 38: 4, 57: 4} | {19: 4, 38: 4, 57: 4} |
| ECG5000 | {28: 4, 56: 4} | {4: 16, 8: 16, 12: 16} | {28: 16} | {28: 16, 56: 16} | {28: 16, 56: 16} |
| ECGFiveDays | {27: 4, 54: 4} | {27: 4, 54: 4} | {27: 4, 54: 4, 81: 4} | {27: 4, 54: 4, 81: 4} | {27: 4, 54: 4, 81: 4} |
| Earthquakes | {102: 4, 204: 4} | {102: 16} | {15: 16} | {15: 4, 30: 4, 45: 4} | {15: 4, 30: 4, 45: 4} |
| ElectricDevices | {2: 8, 4: 8, 6: 8} | {2: 8, 4: 8, 6: 8} | {2: 8, 4: 8, 6: 8} | {2: 8, 4: 8, 6: 8} | {2: 8, 4: 8, 6: 8} |
| FaceAll | {3: 16, 6: 16, 9: 16} | {3: 16, 6: 16, 9: 16} | {3: 16, 6: 16, 9: 16} | {3: 16, 6: 16, 9: 16} | {3: 16, 6: 16, 9: 16} |
| FaceFour | {70: 8} | {70: 16} | {70: 4, 140: 4, 210: 4} | {70: 4, 140: 4, 210: 4} | {70: 4, 140: 4, 210: 4} |
| FacesUCR | {26: 16, 52: 16, 78: 16} | {26: 16, 52: 16} | {26: 16, 52: 16, 78: 16} | {26: 16, 52: 16, 78: 16} | {26: 16, 52: 16, 78: 16} |
| FiftyWords | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} |
| Fish | {92: 8, 184: 8} | {92: 8, 184: 8} | {13: 16, 26: 16, 39: 16} | {92: 16, 184: 16} | {92: 16, 184: 16} |
| FordA | {15: 8, 30: 8, 45: 8} | {100: 8, 200: 8} | {15: 16, 30: 16, 45: 16} | {15: 16, 30: 16, 45: 16} | {15: 16, 30: 16, 45: 16} |
| FordB | {100: 8} | {100: 16} | {15: 16, 30: 16, 45: 16} | {15: 16, 30: 16, 45: 16} | {15: 16, 30: 16, 45: 16} |
| GunPoint | {4: 4, 8: 4} | {4: 8, 8: 8} | {4: 8, 8: 8} | {4: 8, 8: 8} | {4: 8, 8: 8} |
| Ham | {86: 8} | {86: 4, 172: 4} | {86: 4, 172: 4} | {86: 8} | {86: 8} |
| HandOutlines | {541: 8, 1082: 8} | {541: 16} | {541: 16} | {541: 16} | {541: 16} |
| Haptics | {218: 16, 436: 16, 654: 16} | {218: 8, 436: 8, 654: 8} | {218: 16, 436: 16} | {218: 8} | {218: 8} |
| Herring | {102: 4, 204: 4} | {15: 4, 30: 4} | {15: 4, 30: 4, 45: 4} | {15: 4, 30: 4} | {15: 4, 30: 4} |
| InlineSkate | {376: 4, 752: 4} | {376: 16} | {376: 16} | {376: 16} | {376: 16} |
| InsectWingbeatSound | {51: 16, 102: 16} | {51: 16, 102: 16} | {51: 16, 102: 16, 153: 16} | {51: 16, 102: 16, 153: 16} | {51: 16, 102: 16, 153: 16} |
| ItalyPowerDemand | {16: 16} | {16: 16} | {16: 16} | {16: 16} | {16: 16} |
| LargeKitchenAppliances | {21: 16, 42: 16, 63: 16} | {21: 8} | {21: 8, 42: 8, 63: 8} | {21: 8, 42: 8} | {21: 8, 42: 8} |
| Lightning2 | {19: 4, 38: 4, 57: 4} | {19: 8, 38: 8} | {19: 16, 38: 16, 57: 16} | {19: 16, 38: 16, 57: 16} | {19: 16, 38: 16, 57: 16} |
| Lightning7 | {63: 4, 126: 4, 189: 4} | {63: 16, 126: 16, 189: 16} | {63: 8, 126: 8, 189: 8} | {63: 8, 126: 8, 189: 8} | {63: 8, 126: 8, 189: 8} |
| Mallat | {204: 16, 408: 16} | {204: 16, 408: 16} | {204: 8, 408: 8} | {204: 16, 408: 16} | {204: 16, 408: 16} |
| Meat | {13: 8, 26: 8, 39: 8} | {13: 8} | {13: 8} | {13: 8} | {13: 8} |
| MedicalImages | {19: 8, 38: 8, 57: 8} | {19: 8, 38: 8, 57: 8} | {19: 8, 38: 8, 57: 8} | {2: 8, 4: 8, 6: 8} | {2: 8, 4: 8, 6: 8} |
| MiddlePhalanxOutlineAgeGroup | {2: 4, 4: 4, 6: 4} | {2: 4, 4: 4} | {2: 4, 4: 4, 6: 4} | {2: 4, 4: 4, 6: 4} | {2: 4, 4: 4, 6: 4} |
| MiddlePhalanxOutlineCorrect | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} |
| MiddlePhalanxTW | {16: 4} | {16: 8} | {16: 4} | {16: 8} | {16: 8} |
| MoteStrain | {16: 8} | {16: 8} | {16: 8} | {16: 8} | {16: 8} |
| NonInvasiveFetalECGThorax1 | {22: 16, 44: 16, 66: 16} | {22: 16, 44: 16, 66: 16} | {22: 16, 44: 16, 66: 16} | {22: 16, 44: 16, 66: 16} | {22: 16, 44: 16, 66: 16} |
| NonInvasiveFetalECGThorax2 | {22: 16, 44: 16, 66: 16} | {150: 16, 300: 16} | {22: 16, 44: 16, 66: 16} | {22: 16, 44: 16, 66: 16} | {22: 16, 44: 16, 66: 16} |
| OSULeaf | {12: 16, 24: 16, 36: 16} | {12: 16, 24: 16, 36: 16} | {12: 16, 24: 16, 36: 16} | {12: 16, 24: 16, 36: 16} | {12: 16, 24: 16, 36: 16} |
| OliveOil | {17: 16, 34: 16, 51: 16} | {17: 16, 34: 16, 51: 16} | {17: 16, 34: 16, 51: 16} | {17: 16, 34: 16, 51: 16} | {17: 16, 34: 16, 51: 16} |
| PhalangesOutlinesCorrect | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} |
| Phoneme | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} |
| Plane | {4: 4, 8: 4, 12: 4} | {4: 8, 8: 8} | {28: 4, 56: 4} | {28: 4, 56: 4} | {28: 4, 56: 4} |
| ProximalPhalanxOutlineAgeGroup | {2: 8, 4: 8} | {2: 8, 4: 8} | {2: 4} | {2: 4, 4: 4} | {2: 4, 4: 4} |
| ProximalPhalanxOutlineCorrect | {16: 4, 32: 4} | {16: 4, 32: 4, 48: 4} | {16: 4, 32: 4, 48: 4} | {16: 4, 32: 4, 48: 4} | {16: 4, 32: 4, 48: 4} |
| ProximalPhalanxTW | {16: 4, 32: 4, 48: 4} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} | {16: 8, 32: 8, 48: 8} |
| RefrigerationDevices | {21: 4} | {21: 4} | {21: 4} | {21: 4} | {21: 4} |
| ScreenType | {21: 4, 42: 4} | {21: 4, 42: 4, 63: 4} | {21: 8, 42: 8, 63: 8} | {21: 4, 42: 4} | {21: 4, 42: 4} |
| ShapeletSim | {15: 4} | {15: 4} | {15: 4} | {15: 4} | {15: 4} |
| ShapesAll | {15: 16, 30: 16, 45: 16} | {102: 16, 204: 16} | {102: 16, 204: 16, 306: 16} | {15: 16, 30: 16, 45: 16} | {15: 16, 30: 16, 45: 16} |
| SmallKitchenAppliances | {144: 8, 288: 8, 432: 8} | {144: 8, 288: 8} | {144: 8, 288: 8} | {144: 8, 288: 8} | {144: 8, 288: 8} |
| SonyAIBORobotSurface1 | {14: 8} | {14: 8} | {14: 8, 28: 8, 42: 8} | {14: 8, 28: 8} | {14: 8, 28: 8} |
| SonyAIBORobotSurface2 | {13: 4} | {13: 8, 26: 8} | {13: 4, 26: 4} | {13: 8, 26: 8} | {13: 8, 26: 8} |
| StarLightCurves | {30: 16, 60: 16} | {30: 16, 60: 16} | {30: 8, 60: 8} | {30: 16, 60: 16} | {30: 16, 60: 16} |
| Strawberry | {47: 16, 94: 16} | {47: 16, 94: 16, 141: 16} | {47: 16, 94: 16, 141: 16} | {47: 16, 94: 16} | {47: 16, 94: 16} |
| SwedishLeaf | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} |
| Symbols | {79: 16, 158: 16, 237: 16} | {79: 16, 158: 16} | {79: 16, 158: 16} | {79: 16, 158: 16, 237: 16} | {79: 16, 158: 16, 237: 16} |
| SyntheticControl | {12: 4, 24: 4, 36: 4} | {12: 8, 24: 8} | {12: 4, 24: 4} | {12: 8, 24: 8} | {12: 8, 24: 8} |
| ToeSegmentation1 | {8: 16, 16: 16, 24: 16} | {55: 16, 110: 16, 165: 16} | {8: 8, 16: 8, 24: 8} | {8: 16, 16: 16, 24: 16} | {8: 16, 16: 16, 24: 16} |
| ToeSegmentation2 | {68: 4, 136: 4, 204: 4} | {68: 16, 136: 16, 204: 16} | {68: 4, 136: 4, 204: 4} | {68: 8, 136: 8} | {68: 8, 136: 8} |
| Trace | {8: 4} | {8: 4, 16: 4} | {8: 4} | {8: 4} | {8: 4} |
| TwoLeadECG | {16: 4, 32: 4} | {16: 8} | {16: 8, 32: 8} | {16: 8} | {16: 8} |
| TwoPatterns | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} | {25: 16, 50: 16, 75: 16} |
| UWaveGestureLibraryAll | {189: 16, 378: 16, 567: 16} | {189: 16, 378: 16, 567: 16} | {189: 16, 378: 16, 567: 16} | {189: 16, 378: 16, 567: 16} | {189: 16, 378: 16, 567: 16} |
| UWaveGestureLibraryX | {63: 8, 126: 8, 189: 8} | {63: 16, 126: 16} | {63: 8, 126: 8, 189: 8} | {63: 16, 126: 16, 189: 16} | {63: 16, 126: 16, 189: 16} |
| UWaveGestureLibraryY | {63: 16, 126: 16, 189: 16} | {63: 16, 126: 16, 189: 16} | {63: 8, 126: 8, 189: 8} | {63: 16, 126: 16, 189: 16} | {63: 16, 126: 16, 189: 16} |
| UWaveGestureLibraryZ | {63: 8, 126: 8, 189: 8} | {63: 16, 126: 16, 189: 16} | {63: 16, 126: 16, 189: 16} | {63: 16, 126: 16, 189: 16} | {63: 16, 126: 16, 189: 16} |
| Wafer | {30: 8, 60: 8} | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} | {30: 16, 60: 16, 90: 16} |
| Wine | {46: 16, 92: 16, 138: 16} | {46: 16, 92: 16} | {46: 16, 92: 16, 138: 16} | {46: 16, 92: 16, 138: 16} | {46: 16, 92: 16, 138: 16} |
| WordSynonyms | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} | {54: 16, 108: 16, 162: 16} |
| Worms | {180: 16, 360: 16, 540: 16} | {180: 8, 360: 8, 540: 8} | {27: 16, 54: 16} | {180: 16, 360: 16, 540: 16} | {180: 16, 360: 16, 540: 16} |
| WormsTwoClass | {180: 8, 360: 8, 540: 8} | {180: 16, 360: 16} | {180: 16, 360: 16} | {180: 16, 360: 16} | {180: 16, 360: 16} |
| Yoga | {85: 16, 170: 16} | {85: 16, 170: 16, 255: 16} | {85: 16, 170: 16, 255: 16} | {85: 16, 170: 16} | {85: 16, 170: 16} |


Table 4.